# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
%pip install pymongo


Note: you may need to restart the kernel to use updated packages.


In [2]:
 from pymongo import MongoClient
import pandas as pd

import re

 client = MongoClient("mongodb://localhost:27017")

client.list_database_names()   # lista de las bases de datos

['admin', 'companies', 'config', 'local']

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [3]:
db=client.companies   # cliente.base_de_datos
colec=db.companies     # base_de_datos.coleccion

In [4]:
# Your Code
list(colec.find({'name':'Babelgum'},{'name':1,'_id':0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [5]:
# Your Code
list(colec.find({'number_of_employees':{'$gte':5000}},{'name':1}).sort('number_of_employees',1).limit(20))

[{'_id': ObjectId('52cdef7e4bab8bd67529aa77'),
  'name': 'JDS Uniphase Corporation'},
 {'_id': ObjectId('52cdef7d4bab8bd675299e01'), 'name': 'PCH International'},
 {'_id': ObjectId('52cdef7f4bab8bd67529c5d2'), 'name': 'Vcare Call Center'},
 {'_id': ObjectId('52cdef7f4bab8bd67529c4bf'), 'name': 'United Internet'},
 {'_id': ObjectId('52cdef7e4bab8bd67529a4f4'), 'name': 'McAfee'},
 {'_id': ObjectId('52cdef7d4bab8bd67529891b'), 'name': 'Spotify'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b1b5'), 'name': 'Air Astana Airlines'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e49'), 'name': 'Nintendo'},
 {'_id': ObjectId('52cdef7e4bab8bd67529ba2e'),
  'name': 'Hexaware Technologies'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'), 'name': 'Facebook'},
 {'_id': ObjectId('52cdef7e4bab8bd67529a453'), 'name': 'OpenText'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b9dd'), 'name': 'CPM Braxis'},
 {'_id': ObjectId('52cdef7d4bab8bd675298b89'), 'name': 'LSI'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b691'),
  'nam

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [6]:
# Your Code
list(colec.find({'$and':[{'founded_year':{'$gt':1999}},
                         {'founded_year':{'$lte':2005}}]},
                        {'name':1, '_id':0,'founded_year':1}))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'Pando Networks', 'founded_year': 2004},
 {'name': 'Ikan', 'foun

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [7]:
# Your Code
list(colec.find({'$and':[{'ipo.valuation_amount':{'$gt':10e8}},
                         {'founded_year':{'$lte':2010}}]},
                        {'name':1, '_id':0,'ipo.valuation_amount':1}))

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'OpenTable', 'ipo': {'valuation_amount': 1050000000}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000}},
 {'name': 'HomeAway', 'ipo': {'valuation_amount': 3000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Higher One', 'ipo': {'valuation_amount': 1060000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Baidu', 'ipo': {'valuation_amount': 4000000000}},
 {'name': 'Chegg', 'ipo': {'valuation



### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [8]:
# Your Code
list(colec.find({'$and':[{'number_of_employees':{'$lte':1000}},
                         {'founded_year':{'$lte':2005}}]},
                        {'name':1, '_id':0,'founded_year':1,'number_of_employees':1}).sort('number_of_employees',-1).limit(10))

[{'name': 'Sonus Networks', 'number_of_employees': 1000, 'founded_year': 1997},
 {'name': 'SK Net Service Company Ltd',
  'number_of_employees': 1000,
  'founded_year': 2001},
 {'name': 'Gumtree', 'number_of_employees': 1000, 'founded_year': 2005},
 {'name': 'Akamai Technologies',
  'number_of_employees': 1000,
  'founded_year': 1998},
 {'name': 'Omniture', 'number_of_employees': 1000, 'founded_year': 1996},
 {'name': 'Antal International',
  'number_of_employees': 1000,
  'founded_year': 1993},
 {'name': "OC'LIANE", 'number_of_employees': 1000, 'founded_year': 2003},
 {'name': 'FANUC Robotics America',
  'number_of_employees': 1000,
  'founded_year': 1982},
 {'name': 'Telenav', 'number_of_employees': 1000, 'founded_year': 1999},
 {'name': 'Yodle', 'number_of_employees': 1000, 'founded_year': 2005}]

### 6. All the companies that don't include the `partners` field.

In [9]:
# Your Code
list(colec.find({'partners':{'$ne': []}},{'name':1}).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297dba'), 'name': 'Google'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e39'), 'name': 'Parakey'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e7e'), 'name': 'boo-box'},
 {'_id': ObjectId('52cdef7c4bab8bd675297eb0'), 'name': 'Echo'},
 {'_id': ObjectId('52cdef7c4bab8bd675297eb4'), 'name': 'Pickle'},
 {'_id': ObjectId('52cdef7c4bab8bd675297ed3'), 'name': 'Weebly'},
 {'_id': ObjectId('52cdef7c4bab8bd675297ed4'), 'name': 'Yola'},
 {'_id': ObjectId('52cdef7c4bab8bd675297f00'), 'name': 'Docstoc'},
 {'_id': ObjectId('52cdef7c4bab8bd675297f0c'), 'name': 'Alibaba'},
 {'_id': ObjectId('52cdef7c4bab8bd675297f24'), 'name': 'TubeMogul'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [10]:
# Your Code
list(colec.find({'category_code':{'$type':"null"}},{'name':1}).sort('number_of_employees',-1).limit(10))

[{'_id': ObjectId('52cdef7e4bab8bd67529b0fe'),
  'name': 'Nippon Telegraph and Telephone Corporation'},
 {'_id': ObjectId('52cdef7e4bab8bd67529aa51'),
  'name': 'Flextronics International'},
 {'_id': ObjectId('52cdef7e4bab8bd67529bb5c'),
  'name': 'Digital Equipment Corporation'},
 {'_id': ObjectId('52cdef7d4bab8bd675298924'), 'name': 'Cox Enterprises'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b998'), 'name': 'ExxonMobil'},
 {'_id': ObjectId('52cdef7d4bab8bd67529a128'), 'name': 'Guitar Center'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b4a4'),
  'name': 'American Power Conversion'},
 {'_id': ObjectId('52cdef7e4bab8bd67529bc6c'), 'name': 'Gottschalks'},
 {'_id': ObjectId('52cdef7c4bab8bd67529852d'), 'name': 'GoDaddy'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b36f'), 'name': 'Advantest'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [11]:
# Your Code
list(colec.find({'$and':[{'number_of_employees':{'$gte':100}},
                         {'number_of_employees':{'$lte':1000}}]},
                        {'name':1, '_id':0,'number_of_employees':1}))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [14]:
# Your Code
list(colec.find({},{'name':1, '_id':0,'ipo.valuation_amount':1}).sort('ipo.valuation_amount',-1).limit(10))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [19]:
# Your Code
list(colec.find({'number_of_employees':{'$gte':1000}},
                         {'name':1, '_id':0,'number_of_employees':1}).sort('number_of_employees',-1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [20]:
# Your Code
list(colec.find({'founded_month':{'$gte':6}},
                        {'name':1, '_id':0,'founded_month':1}).limit(1000))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Postini', 'founded_month': 6},
 {'name': 'Geni', 'founded_month': 6},
 {'name': 'Fox Interactive Media', 'founded_month': 6},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10},
 {'name': 'Technorati', 'founded_month': 6},
 {'name': 'Sparter', 'founded_month': 6},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12},
 {'name': 'Jingle Networks', 'founded_month': 9},
 {'name': 'LifeLock', 'founded_month': 8},
 {'name': 'Wesabe', 'founded_month': 12},
 {'name': 'SmugMug', 'founded_month': 11},
 {'name': 'Google', 'founded_month': 9},
 {'name': 'Skype', 'founded_month': 8},
 {'name': 'Pando Networks', 'founded_month': 7},
 {'name': 'Ikan', 'founded_month': 12},
 {'name': 'del

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [31]:
# Your Code
list(colec.find({'$and':[{'acquisitions.price_amount':{'$gte':1e7}},
                         {'founded_year':{'$lte':2000}}]},
                        {'name':1, '_id':0}).sort('acquisitions.price_amount',-1).limit(10))

[{'name': 'Sprint Nextel'},
 {'name': 'Hewlett-Packard'},
 {'name': 'Google'},
 {'name': 'Amgen'},
 {'name': 'Vodafone'},
 {'name': 'Viacom'},
 {'name': 'Microsoft'},
 {'name': 'Nokia'},
 {'name': 'Intel'},
 {'name': 'Cisco'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [33]:
# Your Code
list(colec.find({'acquisition.acquired_year':{'$gte':2010}},
                        {'name':1, '_id':0,'acquisitions':1}).sort('acquisitions.price_amount',-1).limit(10))

[{'name': 'Sprint Nextel',
  'acquisitions': [{'price_amount': 35000000000,
    'price_currency_code': 'USD',
    'term_code': None,
    'source_url': 'http://www.theage.com.au/news/breaking/sprint-completes-nextel-acquisition/2005/08/15/1123957976577.html',
    'source_description': 'Sprint completes Nextel acquisition',
    'acquired_year': 2005,
    'acquired_month': 8,
    'acquired_day': 15,
    'company': {'name': 'Nextel Communications', 'permalink': 'nextel'}},
   {'price_amount': 483000000,
    'price_currency_code': 'USD',
    'term_code': 'stock',
    'source_url': 'http://www.foxbusiness.com/story/markets/industries/telecom/sprint-acquire-virgin-mobile--million/',
    'source_description': 'Sprint to Acquire Virgin Mobile for $483M',
    'acquired_year': 2009,
    'acquired_month': 7,
    'acquired_day': 28,
    'company': {'name': 'Virgin Mobile USA', 'permalink': 'virgin-mobile'}},
   {'price_amount': 831000000,
    'price_currency_code': 'USD',
    'term_code': 'cash',
 

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [21]:
# Your Code
list(colec.find({}, {'name':1, '_id':0,'founded_year':1}).sort('founded_year',-1))

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'Tongxue', 'founded_year': 2013},
 {'name': 'Shopseen', 'founded_year': 2013},
 {'name': 'VistaGen Therapeutics', 'founded_year': 2013},
 {'name': 'PeekYou', 'founded_year': 2012},
 {'name': 'headr', 'founded_year': 2012},
 {'name': 'Pinger', 'founded_year': 2012},
 {'name': 'Widgetbox', 'founded_year': 2012},
 {'name': 'Mobiluck', 'founded_year': 2012},
 {'name': 'Skydeck', 'founded_year': 2012},
 {'name': 'Simplicant', 'founded_year': 2012},
 {'name': 'Springleap', 'founded_year': 2012},
 {'name': 'Jumbuck Entertainment', 'founded_year': 2012},
 {'nam

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [36]:
# Your Code
list(colec.find({'founded_day':{'$lte':7}}, {'name':1, '_id':0}).sort('acquisitions.price_amount',-1).limit(10))

[{'name': 'Google'},
 {'name': 'Microsoft'},
 {'name': 'Yahoo!'},
 {'name': 'Juniper Networks'},
 {'name': 'FlowPlay'},
 {'name': 'eBay'},
 {'name': 'IAC'},
 {'name': 'OpenText'},
 {'name': 'Facebook'},
 {'name': 'PayPal'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [37]:
# Your Code
list(colec.find({'$and':[{'category_code':'web'},
                         {'number_of_employees':{'$lte':4000}}]},
                        {'name':1, '_id':0}).sort('number_of_employees',1).limit(10))

[{'name': 'Bebo'},
 {'name': 'Go2Web20'},
 {'name': 'Checkster'},
 {'name': 'Gumiyo'},
 {'name': 'Fox Interactive Media'},
 {'name': 'stylediary'},
 {'name': 'PartnerUp'},
 {'name': 'Sportingo'},
 {'name': 'Ticketmaster'},
 {'name': 'coRank'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [44]:
# Your Code
list(colec.find({'$and':[{'acquisitions.price_amount':{'$gte':10000000}},
                         {'acquisitions.price_currency_code':'EUR'}]},
                        {'name':1, '_id':0}))

[{'name': 'Yahoo!'},
 {'name': 'Conduit'},
 {'name': 'FlowPlay'},
 {'name': 'Nokia'},
 {'name': 'Toluna'},
 {'name': 'SAP'},
 {'name': 'Vodafone'},
 {'name': 'EXFO'},
 {'name': 'Meetic'},
 {'name': 'SolarCity'},
 {'name': 'Hi-media'},
 {'name': 'Populis'},
 {'name': 'Playtech'},
 {'name': 'Vistaprint'},
 {'name': 'Springer'},
 {'name': 'Telefonica'},
 {'name': 'Constellation Software'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [45]:
# Your Code
list(colec.find({'founded_month':{'$lte':3}},
                        {'name':1, '_id':0,'acquisitions':1}).limit(10))

[{'name': 'Facebook',
  'acquisitions': [{'price_amount': None,
    'price_currency_code': 'USD',
    'term_code': 'cash',
    'source_url': 'http://www.techcrunch.com/2007/07/19/breaking-facebook-has-acquired-parakey/',
    'source_description': '',
    'acquired_year': 2007,
    'acquired_month': 7,
    'acquired_day': 1,
    'company': {'name': 'Parakey', 'permalink': 'parakey'}},
   {'price_amount': 47500000,
    'price_currency_code': 'USD',
    'term_code': 'cash_and_stock',
    'source_url': 'http://www.techcrunch.com/2009/08/10/facebook-acquires-friendfeed',
    'source_description': 'Facebook Acquires FriendFeed',
    'acquired_year': 2009,
    'acquired_month': 8,
    'acquired_day': 10,
    'company': {'name': 'FriendFeed', 'permalink': 'friendfeed'}},
   {'price_amount': None,
    'price_currency_code': 'USD',
    'term_code': None,
    'source_url': 'http://techcrunch.com/2010/02/19/octazen-what-the-heck-did-facebook-just-buy-exactly-and-why/',
    'source_description': ''

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [48]:
# Your Code
list(colec.find({'$and':[{'founded_year':{'$gt':2000}},
                         {'founded_year':{'$lte':2010}},
                        {'acquisitions.acquired_year':{'$lte':2011}}]},
                        {'name':1, '_id':0,'founded_year':1}))

[{'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Twitter', 'founded_year': 2006},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Skype', 'founded_year': 2003},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'iSkoot', 'founded_year': 2005},
 {'name': 'iContact', 'founded_year': 2003},
 {'name': 'blinkx', 'founded_year': 2004},
 {'name': 'RockYou', 'founded_year': 2005},
 {'name': 'TheFind', 'founded_year': 2006},
 {'name': 'Meebo', 'founded_year': 2005},
 {'name': 'LinkedIn', 'founded_year': 2003},
 {'name': 'Box', 'founded_year': 2005},
 {'name': 'Carbonite', 'founded_year': 2006},
 {'name': 'Conduit', 'founded_year': 2005},
 {'name': 'GigaOM', 'founded_year': 2006},
 {'name': 'Seesmic', 'founded_year': 2007},
 {'name': 'Oodle', 'founded_year': 2004},
 {'name': 'Vast', 'founded_year': 2001},
 {'name': 'hi5', 'founded_year': 2003},

### 20. All the companies that have been 'deadpooled' after the third year.

In [50]:
# Your Code
(list(colec.find({"deadpooled_year":{'$gt': 3}}, {"name": 1,"_id": 0})))

[{'name': 'Omnidrive'},
 {'name': 'Babelgum'},
 {'name': 'Sparter'},
 {'name': 'Thoof'},
 {'name': 'Mercora'},
 {'name': 'Wesabe'},
 {'name': 'Jangl SMS'},
 {'name': 'Stickam'},
 {'name': 'AllPeers'},
 {'name': 'EQO'},
 {'name': 'AllofMP3'},
 {'name': 'SellABand'},
 {'name': 'Zlio'},
 {'name': 'Jaiku'},
 {'name': 'Spotplex'},
 {'name': 'TripUp'},
 {'name': 'Fleck'},
 {'name': 'Coghead'},
 {'name': 'Zooomr'},
 {'name': 'Swivel'},
 {'name': 'Slide'},
 {'name': 'PlaceBlogger'},
 {'name': 'TripHub'},
 {'name': 'Sway'},
 {'name': 'adBrite'},
 {'name': 'Edgeio'},
 {'name': 'coComment'},
 {'name': 'Hotelicopter'},
 {'name': 'Ujogo'},
 {'name': 'PeerMe'},
 {'name': 'CollegeWikis'},
 {'name': 'Hotswap'},
 {'name': 'SplashCast'},
 {'name': 'Tangler'},
 {'name': 'TalkPlus'},
 {'name': 'Citizenbay'},
 {'name': 'PermissionTV'},
 {'name': 'WebbAlert'},
 {'name': 'Nirvanix'},
 {'name': 'Meetro'},
 {'name': 'MyToons'},
 {'name': 'KnockaTV'},
 {'name': 'Millsberry'},
 {'name': 'Revver'},
 {'name': 'Yoo